In [92]:
# 1.) Get Active Time

# Given a sequence of timestamps & actions of a dasher's activity within a day, we would like to know the 
# active time of the dasher. Idle time is defined as the dasher has NO delivery at hand. (That means all 
# items have been dropped off at this moment and the dasher is just waiting for another pickup) Active time equals
# total time minus idle time.

# Ref: https://leetcode.com/discuss/interview-question/1302606/DoorDash-onsite-interview-(new-question!)



def get_active_time(activity):
    pickups = []
    dropoffs = []
    smallest_pickup = float('inf')
    highest_dropoff = float('-inf')

    for a in activity:
        a = a.split('|')
        a_type = a[1].strip()
        a_mins = get_mins(a[0].strip())

        if a_type == 'pickup':
            pickups.append(a_mins)
            smallest_pickup = min(smallest_pickup, a_mins)
        else:
            dropoffs.append(a_mins)
            highest_dropoff = max(highest_dropoff, a_mins)

    interval = []
    for p, d in zip(pickups, dropoffs):
        interval.append([p, d])

    total_time = highest_dropoff - smallest_pickup
    idle_time = 0
    for idx in range(len(interval) - 1):
        curr = interval[idx]
        nxt = interval[idx + 1]

        if curr[1] < nxt[0]:
            idle_time += nxt[0] - curr[1]
        else:
            nxt[1] = max(nxt[1], curr[1])

    return total_time - idle_time


def get_mins(t):
    t = t.split(':')
    hrs = int(t[0])
    mins = int(t[1][:-2])
    if t[1][-2:] == 'pm':
        if hrs < 12:
            hrs += 12
    else:
        if hrs == 12:
            hrs = 0

    return 60*hrs + mins

#self
def get_active_time2(activity):
    if not activity:
        return 0;
    pre_time = None 
    pre_order = 0 
    active_time = 0
    for a in activity:
        a = a.split('|')
        a_type = a[1].strip()
        a_mins = get_mins(a[0].strip())
        if pre_order > 0:
            active_time += a_mins - pre_time
        if a_type == 'pickup':
            pre_order += 1    
        else:
            pre_order -= 1
        pre_time = a_mins

    

    return active_time

if __name__ == '__main__':
    activity = [
        '8:30am | pickup',
        '9:10am | dropoff',
        '10:20am| pickup',
        '12:15pm| pickup',
        '12:45pm| dropoff',
        '2:25pm | dropoff',
    ]

    idle_time = get_active_time(activity)

    print(idle_time)
    print(get_active_time2(activity))
    
    
    
# 759. Employee Free Time
"""
# Definition for an Interval.
class Interval:
    def __init__(self, start: int = None, end: int = None):
        self.start = start
        self.end = end
"""

class Solution:
    def employeeFreeTime(self, schedule: '[[Interval]]') -> '[Interval]':
        
        events = []
        
        for intervals in schedule:
            for e in intervals:
                events.append((e.start, -1)) # using -1 such that starting working event will be processed first
                events.append((e.end, 1))
                
        
        events.sort()
        count = 0
        res = []
        prev = None
        for time, cnt in events:
                        
            if count == 0 and prev:
                res.append(Interval(prev, time))
            
            count += cnt
            prev = time
        return res

285
285


In [18]:
# 2.) All Shortest Path

# Ref: https://leetcode.com/discuss/interview-question/1353434/Doordash-Phone-Screen-or-Senior-Software-Engineer-or-July-2021

import collections, heapq


def find_shortest_paths(g_nodes, sources, destinations, weights):
    graph = collections.defaultdict(list)
    weight_between = {}
    for s, d, w in zip(sources, destinations, weights):
        graph[s].append(d)
        graph[d].append(s)
        weight_between[(s, d)] = w
        weight_between[(d, s)] = w

    parents = collections.defaultdict(set)

    dist = collections.defaultdict(lambda: float('inf'))
    dist[1] = 0

    def relaxation(v, u, w):  
        if dist[v] > dist[u] + w:
            dist[v] = dist[u] + w
            parents[v] = {u}
        elif dist[v] == dist[u] + w:
            parents[v].add(u)

    q = [(0, 1)]
    relaxed = set()
    while q:
        _, node = heapq.heappop(q)

        for nei in graph[node]:
            if nei not in relaxed:
                relaxation(nei, node, weight_between[(nei, node)])
                heapq.heappush(q, (weight_between[(nei, node)], nei))

        relaxed.add(node)


    shortest_edges = set()
    visited = set()
    dfs(g_nodes, parents, visited, shortest_edges)

    ans = []
    for s, d in zip(sources, destinations):
        if (s, d) in shortest_edges or (d, s) in shortest_edges:
            ans.append('YES')
        else:
            ans.append('NO')
    print(parents)
    print(ans)


def dfs(node, parents, visited, shortest_edges):
    if node in visited:
        return
    visited.add(node)
    while parents[node]:
        p = parents[node].pop()
        shortest_edges.add((node, p))
        dfs(p, parents, visited, shortest_edges)


# self
# Dijkstra output all shortest path
def find_shortest_paths2(n, sources, destinations, distances):
    graph = collections.defaultdict(dict)
    for s, d, dis in zip(sources, destinations, distances):
        graph[s-1][d-1] = dis
        graph[d-1][s-1] = dis
    parent = collections.defaultdict(set)  # shortest path parents
    best_distance = collections.defaultdict(lambda: float('inf'))  # 
    source = 0
    target = n-1
    best_distance[source] = 0
    heap = [(0, 0)]
    consolidated = set()
    while heap:
        _, cur = heapq.heappop(heap)
        for child in graph[cur]:
            if child not in consolidated: 
                # since greed by using heap, we consolidate the shortest reachable nodes already
                # otherwise we need to relax all node / edges
                d = graph[cur][child]
                if best_distance[child] > best_distance[cur] + d:
                    # 0->cur->child < 0->child
                    best_distance[child] = best_distance[cur] + d
                    parent[child] = set([cur])
                elif best_distance[child] == best_distance[cur] + d:
                    parent[child].add(cur)
                heapq.heappush(heap, (d, child))
        consolidated.add(cur)
        
    print(f"shotest distance {source+1} to {target+1} is {best_distance[target]}")    
    edge_in_shortest = set()
    visited = set([target])  
      # the visited is for duplicated calls, not for cycle,
    # there should not be cycles in dijkstra results
    
    def dfs_mark_edge(cur):
        if cur == source:
            return
        for par in parent[cur]:
            edge_in_shortest.add((par+1, cur+1))
            if par not in visited:  # if we need to get all paths,
                # we need to call dfs with memo
                # still need to call get all paths cur + [par -> sources]
                visited.add(par)
                dfs_mark_edge(par)  
                

    # output all paths
    memo = {}
  
    def dfs(cur):
        if cur == source:
            return [[]]
        if cur in memo:
            return memo[cur]
        cur_paths = []
        for par in parent[cur]:
            edge_in_shortest.add((par+1, cur+1))  # mark the edge 
            par_paths = dfs(par)  
            cur_paths.extend([pp+[(par+1, cur+1)] for pp in par_paths])
        memo[cur] = cur_paths
        return cur_paths
    all_paths = dfs(target)
    print(all_paths)
    
    result = []
    for start, end in zip(sources, destinations):
        if (start, end) in edge_in_shortest or (end, start) in edge_in_shortest:
            tag = "YES"
        else:
            tag = "NO"
        result.append(tag)
    return result
    
    


if __name__ == '__main__':
    find_shortest_paths(
        5,
        [1, 2, 3, 4, 5, 1, 5],
        [2, 3, 4, 5, 1, 3, 3],
        [1, 1, 1, 1, 3, 2, 1]
    )
    print(find_shortest_paths2(
        5,
        [1, 2, 3, 4, 5, 1, 5],
        [2, 3, 4, 5, 1, 3, 3],
        [1, 1, 1, 1, 3, 2, 1]
    ))

defaultdict(<class 'set'>, {2: set(), 5: set(), 3: set(), 4: {3}, 1: set()})
['YES', 'YES', 'NO', 'NO', 'YES', 'YES', 'YES']
shotest distance 1 to 5 is 3
[[(1, 5)], [(1, 3), (3, 5)], [(1, 2), (2, 3), (3, 5)]]
['YES', 'YES', 'NO', 'NO', 'YES', 'YES', 'YES']


In [35]:
# 3.) Closest Drivers

# Ref: https://leetcode.com/discuss/interview-question/1293040/Doordash-or-Phone-Screen-or-Software-Engineer-E4-or-Closest-Drivers-to-Restaurant

import random


class Dasher:
    def __init__(self, id, lastLocation, rating):
        self.id = id
        self.lastLocation = lastLocation
        self.rating = rating


class Location:
    def __init__(self, longitude, latitude):
        self.longitude = longitude
        self.latitude = latitude


def GetDashers():
    loc1 = Location(1, 3)
    d1 = Dasher(1, loc1, 4)
    loc2 = Location(2, 2)
    d2 = Dasher(2, loc2, 4)
    loc3 = Location(4, 0)
    d3 = Dasher(3, loc3, 4)
    loc4 = Location(1, 1)
    d4 = Dasher(4, loc4, 5)

    loc5 = Location(4, 3)
    d5 = Dasher(5, loc1, 2)
    loc6 = Location(7, 2)
    d6 = Dasher(6, loc6, 4)
    loc7 = Location(4, 0)
    d7 = Dasher(7, loc3, 4)
    loc8 = Location(1, 1)
    d8 = Dasher(8, loc4, 3)

    return [d1, d2, d3, d4, d5, d6, d7, d8]


def find_k_closest_drivers(k):
    dashers = GetDashers()
    quick_select(0, len(dashers) - 1, dashers, k)

    return dashers[:k]


def quick_select(left, right, dashers, k):
    while left <= right:
        pivot = random.randint(left, right)
        pivot = partition(pivot, left, right, dashers)

        if pivot == k:
            return
        elif pivot < k:
            left = pivot + 1
        else:
            right = pivot - 1


def get_sum_hash(dasher):
    loc = dasher.lastLocation
    return (loc.longitude ** 2 + loc.latitude ** 2, - dasher.rating)


def partition(pivot, left, right, dashers):
    pivot_ele = get_sum_hash(dashers[pivot])
    dashers[right], dashers[pivot] = dashers[pivot], dashers[right]

    store_idx = left
    for idx in range(left, right):
        if get_sum_hash(dashers[idx]) < pivot_ele:
            dashers[store_idx], dashers[idx] = dashers[idx], dashers[store_idx]
            store_idx += 1

    dashers[store_idx], dashers[right] = dashers[right], dashers[store_idx]

    return store_idx


# self
def find_k_closest(k, restaurant):
    def dist(dasher):
        lg = dasher.lastLocation.longitude - restaurant.longitude     
        lt = dasher.lastLocation.latitude - restaurant.latitude
        return lg**2 + lt**2   
    
    dashers = GetDashers()
    dashers = [(dist(d), -d.rating, d.id) for d in dashers]
    
    # heapq.heapify(dashers)
    # result = []
    # for _ in range(k):   # O(N + klg N)
    #     result.append(heapq.heappop(dashers)[2])
    # return result
    
    # or k size max heap, heapify, pushpop for the rest element, get final results in maxheap
    # O(k + n lg K)
    
    
    # quick_select(0, len(dashers) - 1, dashers, k)
    def quick_select2(left, right, k):  # from left idx, to righ idx, find idx k
        while left <= right:
            pivot = random.randint(left, right)
            pivot = partition2(pivot, left, right)

            if pivot == k:
                return
            elif pivot < k:
                left = pivot + 1
            else:
                right = pivot - 1
            
    def partition2(pivot, left, right):
        dashers[right], dashers[pivot] = dashers[pivot], dashers[right]

        first_le = left
        for idx in range(left, right):  
            # if small, equal, large, can not use for loop need while loop, since large swap do not advance idx
            if dashers[idx] < dashers[right]:
                dashers[first_le], dashers[idx] = dashers[idx], dashers[first_le]
                first_le += 1

        dashers[first_le], dashers[right] = dashers[right], dashers[first_le]

        return first_le
    quick_select2(0, len(dashers)-1, k-1)
    return [id for _, _, id in dashers[:k]]
# time complexity 
# https://www.geeksforgeeks.org/advanced-master-theorem-for-divide-and-conquer-recurrences/



if __name__ == '__main__':
    howmany_dashers = 3
    dashers = find_k_closest_drivers(howmany_dashers)

    for dasher in dashers:
        print(
            f'dasher_id: {dasher.id}, rating: {dasher.rating},'
            f'location: {dasher.lastLocation.latitude},'
            f' {dasher.lastLocation.longitude}'
        )
        
    restaurant = Location(0,0)
    print(find_k_closest(3, restaurant))
    
    
    
# use maxheap with k size can solve this as well, during runtime, we can add / delete if it is hashheap

class HashHeap:
    def __init__(self):
        
        self.heap = []   #  (key, value)
        self.hash = {}    # key to index
        self.count = 0
        
    def add(self, key, value):
        self.heap.append((key, value))
        self.hash[key] = self.count
        self._siftUp(self.count)
        self.count += 1
    
    def _swap(self, a, b):  # index
        self.heap[a], self.heap[b] = self.heap[b], self.heap[a]
        self.hash[self.heap[a][0]] = a
        self.hash[self.heap[b][0]] = b
        
    def remove(self, key):
        pos = self.hash[key]
        self._swap(pos, self.count - 1)
        self.heap.pop()
        del self.hash[key]
        self.count -= 1
        if pos < self.count:
            idx = self._siftUp(pos) # this seems not necessary
            self._siftDown(idx)
            
    def _siftUp(self, idx):
        
        while idx > 0:
            parent = (idx - 1) // 2
            if smaller(self.heap[parent], self.heap[idx]):
                break
            
            self._swap(idx, parent)
            idx = parent
        return idx
    
    def _siftDown(self, idx):
        
        while idx < self.count:
            
            left = (2 * idx + 1)
            right = (2 * idx + 2)
            
            minIdx = idx
            
            if left < self.count and smaller(self.heap[left], self.heap[minIdx]):
                minIdx = left
            if right < self.count and smaller(self.heap[right], self.heap[minIdx]):
                minIdx = right
            
            if minIdx == idx:
                break
            
            self._swap(idx, minIdx)
            idx = minIdx
            
        return idx
    
    def peekMin(self):
        
        return self.heap[0]
    
    def pop(self):
        
        result = self.heap[0]
        self.remove(result[0])
        return result
    
    def update(self, key, value):
        
        self.remove(key)
        self.add(key, value)
        
    def __len__(self):
        
        return len(self.heap)
    
    def contains(self, key):
        return key in self.hash
    

dasher_id: 8, rating: 3,location: 1, 1
dasher_id: 4, rating: 5,location: 1, 1
dasher_id: 2, rating: 4,location: 2, 2
[4, 8, 2]


In [39]:
# 4.) Longest Valid Order

# Find longest valid subarray

# Ex 1: orders = ['P1', 'P1', 'D1'], return ['P1', 'D1']
# Ex 2: orders = ['P1', 'P1', 'D1', 'D1'], return ['P1', 'D1']

# Ref: https://leetcode.com/discuss/interview-question/914113/Longest-valid-orders-path-(Doordash)



def longest_valid_order(orders):
    longest_idxs = [0, 0]

    for start in range(len(orders)):
        for end in range(start + 1, len(orders)):
            pickups = set()
            deliveries = set()
            is_valid = True

            for idx in range(start, end + 1):
                order_type = orders[idx][0]
                order_id = orders[idx][1:]

                if order_type == 'P':
                    if order_id not in pickups and order_id not in deliveries:
                        pickups.add(order_id)
                    else:
                        is_valid = False
                        break
                else:
                    if order_id in pickups and order_id not in deliveries:
                        deliveries.add(order_id)
                    else:
                        is_valid = False
                        break

            if is_valid and len(pickups) == len(deliveries):
                update_logest_idxs(longest_idxs, start, end + 1)

    ans = []
    for idx in range(longest_idxs[0], longest_idxs[1]):
        ans.append(orders[idx])

    print(ans)


def update_logest_idxs(longest_idxs, start, end):
    if longest_idxs[1] - longest_idxs[0] < end - start:
        longest_idxs[0] = start
        longest_idxs[1] = end

# self
def valid_path(path):
    open = set()
    closed = set()
    for order in path:
        type = order[0]
        num = order[1:]
        if type == 'P':
            if num in open or num in closed:
                return False 
            else:
                open.add(num)
        elif type == 'D':
            if num in open:
                open.discard(num)  # set discard, dict pop
                closed.add(num)
            else:
                return False
    return True 
        
    
def max_valid_path(path):
    open2idx = {} 
    closed2idx = {}
    start = 0
    max_len = 0
    max_start = 0
    size = len(path)
    while start < size:
        for cur in range(start, size):
            order = path[cur]
            type = order[0]
            num = order[1:]

            if type == 'P':
                if num in open2idx or num in closed2idx:
                    start = (open2idx[num] + 1) if num in open2idx else (open2idx[num] + 1)
                    open2idx.clear()
                    closed2idx.clear()
                    break
                open2idx[num] = cur
            elif type == 'D':
                if num in open2idx:
                    open2idx.pop(num, None)
                    closed2idx[num] = cur
                else:
                    start = cur + 1                        
                    open2idx.clear()
                    closed2idx.clear()
                    break 
            cur_len = cur - start +1
            if len(open2idx) == 0 and cur_len > max_len:
                max_len = cur_len
                max_start = start
        if cur == size:
            break 
    return path[max_start : max_start+max_len] 
    
if __name__ == '__main__':
    orders = ['P1', 'P1', 'P2', 'D3', 'P1', 'P2', 'D2', 'D1', 'P4', 'D3', 'D1']
    longest_valid_order(orders)
    print(valid_path(orders))
    print(max_valid_path(orders))

['P1', 'P2', 'D2', 'D1']
False
['P1', 'P2', 'D2', 'D1']


In [ ]:
# 5.) Median finder with followups:

# find running median of a data stream
# case 1: data stream contains only nums btw 0 and 100
# case 2: 99% nums are btw 0 to 100

# Ref: https://leetcode.com/problems/find-median-from-data-stream/

class MedianSearch:
    def __init__(self):
        self.middle = [0] * 101
        self.middle_ct = 0
        self.left = []
        self.right = []

    def add_num(self, num):
        if num < 0:
            self.edge_insert(self.left, num)
        elif num > 100:
            self.edge_insert(self.right, num)
        else:
            self.middle[num] += 1
            self.middle_ct += 1

    def add_num2(self, num):
        self.middle[num] += 1
        self.middle_ct += 1

    def get_median2(self):
        first = second = None
        if self.middle_ct % 2 != 0:
            first = (self.middle_ct // 2) + 1
        else:
            first = (self.middle_ct // 2)
            second = first + 1

        first_val = second_val = None
        curr = 0
        ith = 0
        while ith < len(self.middle):
            curr += self.middle[ith]
            if curr >= first:
                first_val = ith
                if second is not None:
                    if curr >= second:
                        second_val = first_val
                    else:
                        ith += 1
                        while ith < len(self.middle):
                            curr += self.middle[ith]
                            if curr >= second:
                                second_val = ith
                                break
                            ith += 1
                break
            ith += 1

        if second is None:
            return first_val
        else:
            return (first_val + second_val) / 2


    def edge_insert(self, arr, num):
        arr.append(num)
        i = len(arr) - 1
        while i > 0 and arr[i - 1] > num:
            arr[i - 1], arr[i] = arr[i], arr[i - 1]
            i -= 1

    def get_median(self):
        total_nums = self.middle_ct + len(self.left) + len(self.right)
        first = (total_nums // 2) + 1
        second = None

        if total_nums % 2 == 0:
            second = first - 1

        first_val = self.find_idx(first)
        if second is not None:
            second_val = self.find_idx(second)
        else:
            return first_val

        return (first_val + second_val) / 2

    def find_idx(self, ith):
        if ith <= len(self.left):
            return self.left[ith - 1]
        elif ith > len(self.left) + self.middle_ct:
            return self.right[ith - len(self.left) - self.middle_ct - 1]
        else:
            curr_ct = len(self.left)
            for ith_val in range(0, 101):
                if self.middle[ith_val] + curr_ct >= ith:
                    return ith_val
                curr_ct += self.middle[ith_val]


if __name__ == '__main__':
    ms = MedianSearch()

    input = [
        ('add_num', 5), ('add_num', 0), ('get_median'), ('add_num', 99),
        ('add_num', 50), ('add_num', 20), ('get_median'), ('add_num', 19),
        ('add_num', 5), ('add_num', 95), ('get_median'), ('add_num', 0),
        ('add_num', 100), ('add_num', 20), ('get_median'), ('add_num', 19),
    ]

    queries = [
        "addNum","findMedian","addNum","findMedian","addNum",
        "findMedian","addNum","findMedian","addNum","findMedian",
        "addNum", "findMedian", "addNum", "findMedian","addNum", "findMedian"
    ]
    values = [[-1],[],[-2],[],[-3],[],[-4],[],[-5],[]]

    values = [
        [-1],[],[2],[],[3],[],[4],[],[5],[],
        [100], [], [33], [], [333], []
    ]

    values = [
        [1],[],[-2],[],[-3],[],[4],[],[5],[],
        [100], [], [332], [], [133], []
    ]
    # values = [
    #     [1],[],[2],[],[3],[],[4],[],[5],[],
    #     [100], [], [33], [], [33], []
    # ]

    # queries = ["addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian","addNum","findMedian"]

    # values = [[155],[],[66],[],[114],[],[0],[],[60],[],[73],[],[109],[],[26],[],[154],[],[0],[],[107],[],[75],[],[9],[],[57],[],[53],[],[6],[],[85],[],[151],[],[12],[],[110],[],[64],[],[103],[],[42],[],[103],[],[126],[],[3],[],[88],[],[142],[],[79],[],[88],[],[147],[],[47],[],[134],[],[27],[],[82],[],[95],[],[26],[],[124],[],[71],[],[79],[],[130],[],[91],[],[131],[],[67],[],[64],[],[16],[],[60],[],[156],[],[9],[],[65],[],[21],[],[66],[],[49],[],[108],[],[80],[],[17],[],[159],[],[24],[],[90],[],[79],[],[31],[],[79],[],[113],[],[39],[],[54],[],[156],[],[139],[],[8],[],[90],[],[19],[],[10],[],[50],[],[89],[],[77],[],[83],[],[13],[],[3],[],[71],[],[52],[],[21],[],[50],[],[120],[],[159],[],[45],[],[22],[],[69],[],[144],[],[158],[],[19],[],[109],[],[52],[],[50],[],[51],[],[62],[],[20],[],[22],[],[71],[],[95],[],[47],[],[12],[],[21],[],[32],[],[17],[],[130],[],[109],[],[8],[],[61],[],[13],[],[48],[],[107],[],[14],[],[122],[],[62],[],[54],[],[70],[],[96],[],[11],[],[141],[],[129],[],[157],[],[136],[],[41],[],[40],[],[78],[],[141],[],[16],[],[137],[],[127],[],[19],[],[70],[],[15],[],[16],[],[65],[],[96],[],[157],[],[111],[],[87],[],[95],[],[52],[],[42],[],[12],[],[60],[],[17],[],[20],[],[63],[],[56],[],[37],[],[129],[],[67],[],[129],[],[106],[],[107],[],[133],[],[80],[],[8],[],[56],[],[72],[],[81],[],[143],[],[90],[],[0],[]]

    for i in range(len(queries)):
        query = queries[i]

        if query == 'addNum':
            value = values[i][0]
            ms.add_num(value)
        else:
            print(ms.get_median())
            
            
# self        
class MedianFinder:

    def __init__(self):
        self.minHeap = []  # larger half numbers
        self.maxHeap = []  # smaller half numbers
        
    def addNum(self, num: int) -> None:
        num = -heapq.heappushpop(self.maxHeap, -num)
        heapq.heappush(self.minHeap, num)
        if len(self.minHeap) > len(self.maxHeap):
            heapq.heappush(self.maxHeap, -heapq.heappop(self.minHeap))
    
    def findMedian(self) -> float:
        if len(self.minHeap) == len(self.maxHeap):
            return (self.minHeap[0] - self.maxHeap[0] )/ 2
        else:
            return -self.maxHeap[0]

In [42]:
# 6.) Nearest City

# Ref: https://leetcode.com/discuss/interview-question/1379696/DoorDASH-Onsite

import collections, random


def find_nearest_cities(x_list, y_list, cities, query_cities):
    xi = collections.defaultdict(list)
    yi = collections.defaultdict(list)
    city_cords = {}
    for x, y, c in zip(x_list, y_list, cities):
        xi[x].append((y, c))
        yi[y].append((x, c))
        city_cords[c] = (x, y)

    for xk in xi.keys():
        xi[xk].sort()

    for yk in yi.keys():
        yi[yk].sort()

    ans = []
    for q_city in query_cities:
        if q_city not in city_cords:
            ans.append(None)

        x, y = city_cords[q_city]
        nearest_city = get_nearest_city(xi, yi, x, y, q_city)
        ans.append(nearest_city)

    return ans


def get_nearest_city(xi, yi, x, y, q_city):
    mins = {
        'city': None,
        'dist': float('inf'),
    }

    find_mins(0, len(xi[x]) - 1, xi[x], y, q_city, mins)
    find_mins(0, len(yi[y]) - 1, yi[y], x, q_city, mins)

    return mins['city']


def find_mins(left, right, axis_n_cities, axis_to_compare, q_city, mins):
    while left <= right:
        mid = random.randint(left, right)
        mid_city = axis_n_cities[mid][1]
        mid_axis = axis_n_cities[mid][0]

        if mid_city == q_city:
            if mid > 0:
                mid_city = axis_n_cities[mid - 1][1]
                mid_axis = axis_n_cities[mid - 1][0]
                mid_dist = abs(axis_to_compare - mid_axis)
                update_mins(mid_dist, mid_city, mins)

            if mid < len(axis_n_cities) - 1:
                mid_city = axis_n_cities[mid + 1][1]
                mid_axis = axis_n_cities[mid + 1][0]
                mid_dist = abs(axis_to_compare - mid_axis)
                update_mins(mid_dist, mid_city, mins)

            break

        if mid_axis < axis_to_compare:
            left = mid + 1
        else:
            right = mid - 1


def update_mins(mid_dist, mid_city, mins):
    if mid_dist < mins['dist']:
        mins['dist'] = mid_dist
        mins['city'] = mid_city
    elif mid_dist ==  mins['dist']:
        mins['city'] = min(mins['city'], mid_city)




# self
import bisect
def find_nearest_cities2(x_list, y_list, cities, query_cities):
    x2yc = collections.defaultdict(list)
    y2xc = collections.defaultdict(list)
    c2xy = {}
    for x, y, c in zip(x_list, y_list, cities):
        # x2yc[x].append((y, c))
        # y2xc[y].append((x, c))
        bisect.insort_right(x2yc[x], (y,c))
        bisect.insort_right(y2xc[y], (x,c))
        c2xy[c] = (x, y)

    # for xk in x2yc.keys():
    #     x2yc[xk].sort()

    # for yk in y2xc.keys():
    #     y2xc[yk].sort()

    ans = []
    for q_city in query_cities:
        if q_city not in c2xy:
            ans.append("NONE")
        
        x, y = c2xy[q_city]
        candidate_cities = find_nearest(x2yc[x], y, q_city)
        candidate_cities.extend(find_nearest(y2xc[y], x, q_city))
        if candidate_cities:  # (distance, city_name)
            candidate_cities.sort()
            ans.append(candidate_cities[0][1])
        else: 
            ans.append("NONE")

    return ans

def find_nearest(arr, coord, city):
    idx = bisect.bisect_left(arr, (coord, city))  # idx is the index for this city
    result = []
    for i in idx-1, idx+1:
        if i < 0 or i > len(arr)-1:
            continue 
        dis = abs(arr[i][0] - coord)
        result.append((dis, arr[i][1]))  # (distance, city)
    return result 
            
        
        

if __name__ == '__main__':
    cities = ['axx', 'axy', 'az', 'axd', 'aa', 'abc', 'abs']
    xs = [0, 1, 2, 4, 5, 0, 1]
    ys = [1, 2, 5 ,3, 4, 2, 0]

    query_cities = ['axx', 'axy', 'abs']

    nearest_cities = find_nearest_cities(xs, ys, cities, query_cities)
    print(nearest_cities)
    print(find_nearest_cities2(xs, ys, cities, query_cities))

['abc', 'abc', 'axy']
['abc', 'abc', 'axy']


In [48]:
# 7.) Pickup & Delivery Permutaions

# print all valid order paths e.g.: delivery after pickup given n

def get_all_patterns(n):
    picked_up = set()
    delivered = set()
    patterns = []
    pattern = []

    find_pattern(picked_up, delivered, pattern, patterns, n)

    return patterns


def find_pattern(picked_up, delivered, pattern, patterns, n):
    if len(pattern) == n * 2:
        patterns.append('->'.join(pattern))
    else:
        for task in range(1, n + 1):
            pickup = 'P' + str(task)
            delivery = 'D' + str(task)

            if pickup not in picked_up:
                picked_up.add(pickup)
                pattern.append(pickup)
                find_pattern(picked_up, delivered, pattern, patterns, n)
                pattern.pop()
                picked_up.remove(pickup)

            if pickup in picked_up and delivery not in delivered:
                delivered.add(delivery)
                pattern.append(delivery)
                find_pattern(picked_up, delivered, pattern, patterns, n)
                pattern.pop()
                delivered.remove(delivery)

# self
def get_all(n):
    def dfs(path):
        if len(path) == n*2:
            result.append("->".join(path))
            return 
        for i in range(1, n+1):
            if i not in path_dict:
                path.append("P" + str(i))
                path_dict[i] = 1
                dfs(path)
                path_dict.pop(i, None)
                path.pop()
            elif path_dict[i] == 1:
                path.append("D" + str(i))
                path_dict[i] = 2
                dfs(path)
                path_dict[i] = 1
                path.pop()
    
    path_dict = {}
    result = []            
    dfs([])
    return result 

if  __name__ == '__main__':
    print(get_all_patterns(2))
    print(get_all(2))

# Time:
# O((N) ^ 2*N)
# Space:
# O(2* N)  => O(N) to hold a single pattern
#  2n -1                         5     3  1
# for p: N! for d => p1, p2..pn-1..pn

#   2n-1   2n - 3 ..... 3, 1
# p1     p2....p3..pn


# 1359. Count All Valid Pickup and Delivery Options

class Solution:
    def countOrders(self, n: int) -> int:

        def factorial(x):
            if x <= 1:
                return 1
            return x * factorial(x-1)
        
        return factorial(2*n)//(2**n) % (10**9 +7)
    
# (a*b +c) * d %b= a*b*d%b + d*c%b = 
# a/

['P1->D1->P2->D2', 'P1->P2->D1->D2', 'P1->P2->D2->D1', 'P2->P1->D1->D2', 'P2->P1->D2->D1', 'P2->D2->P1->D1']
['P1->D1->P2->D2', 'P1->P2->D1->D2', 'P1->P2->D2->D1', 'P2->P1->D1->D2', 'P2->P1->D2->D1', 'P2->D2->P1->D1']


In [51]:
# 8.) Stream last x max

# Given a streaming data of the form (timestamp, value),
# find the maximum value in the stream in the last X seconds.

# Assume time is monotonically increasing.
# Assume time is in the order of seconds.
# max_value() function finds the max in the last X seconds.

# Ref: https://leetcode.com/discuss/interview-question/1302614/DoorDash-Onsite-Interview-(new-question-again!)

import collections


class StreamProcessor:
    def __init__(self, x):
        self.x = x
        self.deque = collections.deque()

    def set_value(self, t, v):
        while self.deque and t - self.deque[0][0] > self.x:
           self.deque.popleft()

        while self.deque and self.deque[-1][1] < v:
            self.deque.pop()

        self.deque.append((t, v))

    def max_value(self, cur_t): # this will be always current time
        while self.deque and cur_t - self.deque[0][0] > self.x:
            self.deque.popleft()

        if not self.deque:
            return -1

        return self.deque[0][1]


# self
import collections
class StreamProcessor2:
    def __init__(self, x):
        self.x = x
        self.q = collections.deque() 


    def set_value(self, t, v):
        self._update(t)
        while self.q and self.q[-1][1] <= v:
            self.q.pop() 
        self.q.append((t,v))
            

    def max_value(self, cur_t): # this will be always current time
        self._update(cur_t)
        if self.q:
            return self.q[0][1]
        else:
            -1

    def _update(self, t):
        
        while self.q and t-self.q[0][0]>self.x:
            self.q.popleft() 


if __name__ == '__main__':
    sp = StreamProcessor(5)
    sp.set_value(0, 5)
    sp.set_value(1, 6)
    sp.set_value(2, 4)
    sp.set_value(5, 5)
    sp.set_value(9, 19)
    sp.set_value(15, 4)
    sp.set_value(16, 25)
    sp.set_value(19, 6)
    sp.set_value(20, 4)

    print(sp.max_value(22))
    
    sp2 = StreamProcessor2(5)
    sp2.set_value(0, 5)
    sp2.set_value(1, 6)
    sp2.set_value(2, 4)
    sp2.set_value(5, 5)
    sp2.set_value(9, 19)
    sp2.set_value(15, 4)
    sp2.set_value(16, 25)
    sp2.set_value(19, 6)
    sp2.set_value(20, 4)

    print(sp2.max_value(22))

6
6


In [ ]:
# 9.) Time increament

# Ref: https://leetcode.com/discuss/interview-question/1387937/Doordash-new-Q

# def get_increments(times):
#     days = {
#         'mon': 1,
#         'tue': 2,
#         'wed': 3,
#         'thu': 4,
#         'fri': 5,
#         'sat': 6,
#         'sun': 7,
#     }

#     start = clean_time(times[0], days)
#     end = clean_time(times[1], days, True)

#     end_borders = set()
#     for minutes in range(1, 6):
#         end_borders.add(add_delta(end, minutes))

#     ans = []
#     curr = start
#     while curr not in end_borders:
#         ans.append(int(curr))
#         curr = add_delta(curr, 5)

#     print(ans)


# def clean_time(t, days, is_end=False):
#     t = t.split()
#     day = days[t[0]]
#     hrs_mins = t[1].split(':')
#     hrs = int(hrs_mins[0])
#     mins = int(hrs_mins[1])

#     if t[2] == 'pm':
#         if hrs < 12:
#             hrs += 1
#     else:
#         if hrs == 12:
#             hrs = 0

#     nearest = mins % 5
#     if nearest < 3:
#         mins -= nearest
#     else:
#         if not is_end:
#             return add_delta(
#                 str(day) + get_str(hrs) + get_str(mins), 5 - nearest
#             )

#     return str(day) + get_str(hrs) + get_str(mins)


# def add_delta(t, delta_mins):
#     day = int(t[0])
#     hrs = int(t[1:3])
#     mins = int(t[3:])

#     if mins + delta_mins < 60:
#         mins += delta_mins
#         return str(day) + get_str(hrs) + get_str(mins)
#     else:
#         mins = 0
#         if hrs + 1 < 24:
#             hrs += 1
#             return str(day) + get_str(hrs) + get_str(mins)
#         else:
#             hrs = 0
#             if day + 1 < 8:
#                 day += 1
#                 return str(day) + get_str(hrs) + get_str(mins)
#             else:
#                 day = 1
#                 return str(day) + get_str(hrs) + get_str(mins)


# def get_str(i):
#     if i < 10:
#         return '0' + str(i)
#     return str(i)

# self
def get_increments2(times):  # all 5 min intervals
    
    if not validate(times):
        return None
    start_day, start_hr, start_min = parse(times[0], True)
    end_day, end_hr, end_min = parse(times[1], False)
    answer = []
    if not start_day or not end_day:
        return None
    while (end_day, end_hr, end_min) != (start_day, start_hr, start_min):
        start_min += 5
        start_day, start_hr, start_min = cleanup(start_day, start_hr, start_min)
        answer.append(format_time(start_day, start_hr, start_min))

    return answer

def validate(times):
    if not times:
        return False 
    if len(times) != 2:
        return False
    return True   
    

def parse(time, is_start):
    days = {
        'mon': 1,
        'tue': 2,
        'wed': 3,
        'thu': 4,
        'fri': 5,
        'sat': 6,
        'sun': 7,
    }
    if len(time.split()) != 3:
        return None, None, None
    day, time, ampm = time.split()
    if day not in days:
        return None, None, None
    if len(time.split()) != 2:
        return None, None, None
    hr, min = time.split(':')
    day = days[day]
    min = int(min)
    if min < 0 or min > 59:
        return None, None, None
    hr = int(hr)
    if hr < 0 or hr > 12:
        return None, None, None
    if ampm == 'pm':
        if hr < 12:
            hr += 12
    elif ampm == 'am':
        if hr == 12:
            hr = 0
    if min % 5 != 0:
        mod = min % 5
        min += (-mod) if is_start else (-mod) # start before real start, end is real end
    else:
        min -= 5 if is_start else 0
    return cleanup(day, hr, min)

def cleanup(day, hr, min):
    if min >= 60:
        hr += min // 60
        min = min % 60
    if hr >= 24:
        day += hr // 24
        hr = hr % 24
    if day -1 >= 7:
        day = (day-1) % 7 +1
    return day, hr, min
    
def format_time(day, hr, min):
    num = min + hr * 100 + day * 10000
    return str(num)

if __name__ == '__main__':
    times = ['tue 00:29 am', 'mon 11:56 pm']
    # get_increments(times)
    print(get_increments2(times))


In [59]:
# 10.) Valid order

# given an pickup delivery, return whether it is valid or not

def validate(pattern):
    picked_up = set()
    delivered = set()

    for task in pattern:
        task_type = task[0]
        task_id = task[1:]
        if task_type == 'P':
            if task_id not in picked_up:
                picked_up.add(task_id)
            else:
                return False
        else:
            if task_id not in delivered and task_id in picked_up:
                delivered.add(task_id)
            else:
                return False


    return len(picked_up) == len(delivered)

import collections
def validate2(pattern):
    order_status = collections.defaultdict(int)
    for p in pattern:
        act, num = p[0], p[1:]
        if act == "P":
            if order_status[num] != 0:
                return False
            order_status[num] = 1
        elif act == "D":
            if order_status[num] != 1:
                return False
            order_status[num] = 2
    return True
            
    
if __name__ == '__main__':
    pattern = [
        'P1', 'P3', 'P2', 'D3', 'P4', 'P404', 'D2', 'D1', 'D404', 'D4',
        'P33', 'D33',
    ]

    print(validate(pattern))
    
    print(validate2(pattern))

True
True


In [74]:
# https://leetcode.com/discuss/interview-question/392780/Doordash-or-Phone-Screen-or-Longest-path-duplicate-numbers-within-a-Matrix


# Given an integer matrix, find the length of the longest path that have same values. The matrix has no boundary limits. 
# (like, Google Maps - see edit for context)

# From each cell, you can either move to two directions: horizontal or vertical. 
# You may NOT move diagonally or move outside of the boundary.

# nums = [
# [1,1],
# [5,5],
# [5,5]
# ]

# Return 4 ( Four 5's).

def find_longest(nums):
    if not nums:
        return 0
    if not nums[0]:
        return 0
    H = len(nums)
    W = len(nums[0])
    result_len = [0]
    result_paths = []
    result_path_set = set()  # remove same path: head to tail vs tail to head
    
    def dfs(i, j, path, visited):
        if len(path) > result_len[0]:
            result_len[0] = len(path)
            result_paths.clear()
            result_path_set.clear()
            # print(f"path > : {path}")
            key = tuple(sorted(path))
            result_path_set.add(key)
            result_paths.append('->'.join([f"({x},{y}):{nums[x][y]}" for x,y in path]))  
        elif len(path) == result_len[0]:
            key = tuple(sorted(path))
            # print(f"path = : {path}")
            # print(f"key = : {key}")
            if key not in result_path_set:
                result_path_set.add(key)
                result_paths.append('->'.join([f"({x},{y}):{nums[x][y]}" for x,y in path]))    
        for di , dj in (-1, 0), (1,0), (0,-1), (0,1):
            ni = (i + di) % H  # no boundaries, round back
            nj = (j + dj) % W
            if nums[i][j] == nums[ni][nj] and (ni, nj) not in visited:
                visited.add((ni, nj))
                path.append((ni,nj))
                dfs(ni, nj, path, visited)
                visited.discard((ni, nj))
                path.pop()
                

    for i in range(H):
        for j in range(W):
            dfs(i, j, [], set())
    
    
    print(result_paths)     
    
    print(result_path_set)          

nums = [
    [1,3,3],
    [0,4,0],
    [0,2,3],   
]
find_longest(nums)

['(2,2):3->(0,2):3->(0,1):3', '(2,0):0->(1,0):0->(1,2):0']
{((0, 1), (0, 2), (2, 2)), ((1, 0), (1, 2), (2, 0))}


In [95]:
# You have a list of intervals representing meeting times.
# You want to schedule another meeting that doesn't overlap with any other meeting, 
# and this meeting is at least k minutes long. Return an array of intervals representing all possible times for this new meeting.

# Follow up: Given a sorted list of meetings. Determine if you can schedule a a new meeting with a given start time and end time. 
# (O(log(n) solution)

# if sorted by start time, then end time, bisect.bisect_left(meetings, (target_end, target_end)),
# check idx -1 and idx if can fit target meeting
# if sorted by end time, then start time, bisect.bisect_right(meeting, (targe_start, target_start)),
# check idx for start time if < target_end, good to go, otherwise wrong

def find_intervals(intervals, k):
    points = []
    for start, end in intervals:
        points.append((start, 1))
        points.append((end, -1))
    points.sort()
    cnt = 0
    pre_time = -1
    result = []
    for time, delta in points:
        if pre_time != -1 and cnt == 0 and time - pre_time >= k:
            result.append((pre_time, time))
        cnt += delta
        pre_time = time
    return result
        
intervals = [(1,5), (3, 10), (4, 13), (14, 15), (18, 19)]
print(find_intervals(intervals, 1))




# DOORDASH: Available Time Problem

# *Google Calendar, Outlook, iCal has been banned from your company! 
# So an intrepid engineer has decided to roll their own implementation. 
# Unfortunately one major missing feature is the ability to find out what time slots are free for a particular individual.

# Given a list of time blocks where a particular person is already booked/busy,
# a start and end time to search between, a minimum duration to search for, 
# find all the blocks of time that a person is free for a potential meeting that will last the aforementioned duration.

# Given: starttime, endtime, duration, meetingslist -> suggestedmeetingtimes

# Let's assume we abstract the representation of times as simple integers, 
# so a valid time is any valid integer supported by your environment. Here is an example input:

# meetingslist: [3,20], [-2, 0], [0,2], [16,17], [19,23], [30,40], [27, 33]

# starttime: -5

# endtime: 27

# minduration: 2

# expected answer:

# freetime: [-5, -2], [23,27]*


def find_free_time(start_time, end_time, meetings, min_duration):
    points = []
    points.append((start_time, 1))
    points.append((start_time, -1))
    points.append((end_time, 1))
    points.append((end_time, -1))
    pre_time = None
    result = []
    count = 0
    for start, end in meetings:
        points.append((start, 1))
        points.append((end, -1))
    points.sort()
    for time, cnt in points:
        if pre_time and count == 0 and time - pre_time >= min_duration:
            result.append((pre_time, time))
        pre_time = time
        count += cnt             
    return result 

print(find_free_time(-5, 27, [[3,20], [-2, 0], [0,2], [16,17], [19,23], [30,40], [27, 33]], 2))
            

# [(-5, -2), (23, 27)]


[(13, 14), (15, 18)]
[(-5, -2), (23, 27)]


In [ ]:
# 695. Max Area of Island


# class Solution:
#     def maxAreaOfIsland(self, grid: List[List[int]]) -> int:
#         max_count = 0
#         visited = set()
#         H = len(grid)
#         W = len(grid[0])
        
#         def dfs(i, j):
#             if i < 0 or i > H-1 or j<0 or j>W-1 or grid[i][j] == 0 or (i,j) in visited:
#                 return 0
#             visited.add((i,j))
#             return 1 + dfs(i+1, j) + dfs(i, j+1) + dfs(i-1, j) + dfs(i, j-1)
        
#         for i in range(H):
#             for j in range(W):
#                 max_count = max(max_count, dfs(i, j))
                
#         return max_count
            
            
#  200. Number of Islands    
class Solution:
    def numIslands(self, grid):
        count = 0
        visited = set()
        H = len(grid)
        W = len(grid[0])
        
        def dfs(i, j):
            if i < 0 or i > H-1 or j<0 or j>W-1 or (grid[i][j] == "0") or ((i,j) in visited):
                return 0
            visited.add((i,j))
            # print(f"visited {i}:{j} ")
            dfs(i+1, j)
            dfs(i, j+1)
            dfs(i-1, j)
            dfs(i, j-1)
            return 1
        
        for i in range(H):
            for j in range(W):
                count += dfs(i, j)
                # print(f"i:{i},j:{j},visit:{visited}")
                
        return count       
    
grid = [["1","1","0","0","0"],
        ["1","1","0","0","0"],
        ["0","0","1","0","0"],
        ["0","0","0","1","1"]]

s = Solution()
print(s.numIslands(grid))
            
               
        

In [ ]:


# 207. Course Schedule


class Solution:
    def canFinish(self, numCourses: int, prerequisites: List[List[int]]) -> bool:
        
        if numCourses <= 0:
            return True
        
        G = [[] for _ in range(numCourses)]
        indegree = [0] * numCourses
        for e in prerequisites:
            G[e[1]].append(e[0])
            indegree[e[0]] += 1
            
        
        queue = collections.deque()
        
        for i, d in enumerate(indegree):
            if indegree[i] == 0:
                queue.append(i)
        
        count = 0
        while queue:
            
            node = queue.popleft()
            count += 1
            for nb in G[node]:
                indegree[nb] -= 1
                if indegree[nb] == 0:
                    queue.append(nb)
                    
        return count == numCourses

### 210. Course Schedule II

class Solution:
    def findOrder(self, numCourses: int, prerequisites: List[List[int]]) -> List[int]:

        if numCourses <= 0:
            return []
        
        G = [[] for _ in range(numCourses)]
        
        indegree = [0] * numCourses
        for e in prerequisites:
            G[e[1]].append(e[0])
            indegree[e[0]] += 1
            
        queue = collections.deque()
        
        for i in range(numCourses):
            if indegree[i] == 0:
                queue.append(i)
        
        res = []
        while queue:
            
            c = queue.popleft()
            res.append(c)
            
            
            for nb in G[c]:
                indegree[nb] -= 1
                if indegree[nb] == 0:
                    queue.append(nb)
                    
                    
        return res if len(res) == numCourses else []

### 210. Course Schedule II (output all solutions)

class Solution:
    def findOrder(self, n: int, prerequisites: List[List[int]]) -> List[int]:
        G = [[] for _ in range(n)]
        indegree = [0] * n
        for e in prerequisites:
            indegree[e[0]] += 1
            G[e[1]].append(e[0])
        # queue = collections.deque()
        # for i in range(n):
        #     if indegree[i] == 0:
        #         queue.append(i)
        res = []
        
        # visited here is critical
        def backTrack(temp, visited):
            
            if len(temp) == n:  # no cycle, with result
                res.append(list(temp))
            else:
                
                for i in range(n):
                    if indegree[i] == 0 and i not in visited:
                        visited.add(i)
                        for nb in G[i]:
                            indegree[nb] -= 1
                        temp.append(i)
                        backTrack(temp, visited)
                        temp.pop()
                        for nb in G[i]:
                            indegree[nb] += 1
                        visited.discard(i)
        backTrack([], set())
        return res
    
    

In [91]:
# 36. Valid Sudoku


class Solution:
    def isValidSudoku(self, board):
        N = 9

        # Use hash set to record the status
        rows = [set() for _ in range(N)]
        cols = [set() for _ in range(N)]
        boxes = [set() for _ in range(N)]

        for r in range(N):
            for c in range(N):
                val = board[r][c]
                # Check if the position is filled with number
                if val == ".":
                    continue

                # Check the row
                if val in rows[r]:
                    return False
                rows[r].add(val)

                # Check the column
                if val in cols[c]:
                    return False
                cols[c].add(val)

                # Check the box
                idx = (r // 3) * 3 + c // 3
                if val in boxes[idx]:
                    return False
                boxes[idx].add(val)

        return True
    
    
class Solution:
    def isValidSudoku(self, board):
        N = 9

        # Use an array to record the status
        rows = [[0] * N for _ in range(N)]
        cols = [[0] * N for _ in range(N)]
        boxes = [[0] * N for _ in range(N)]

        for r in range(N):
            for c in range(N):
                # Check if the position is filled with number
                if board[r][c] == ".":
                    continue

                pos = int(board[r][c]) - 1

                # Check the row
                if rows[r][pos] == 1:
                    return False
                rows[r][pos] = 1

                # Check the column
                if cols[c][pos] == 1:
                    return False
                cols[c][pos] = 1

                # Check the box
                idx = (r // 3) * 3 + c // 3
                if boxes[idx][pos] == 1:
                    return False
                boxes[idx][pos] = 1

        return True
    
# 37. Sudoku Solver

from collections import defaultdict
class Solution:
    def solveSudoku(self, board):
        """
        :type board: List[List[str]]
        :rtype: void Do not return anything, modify board in-place instead.
        """
        def could_place(d, row, col):
            """
            Check if one could place a number d in (row, col) cell
            """
            return not (d in rows[row] or d in columns[col] or \
                    d in boxes[box_index(row, col)])
        
        def place_number(d, row, col):
            """
            Place a number d in (row, col) cell
            """
            rows[row][d] += 1
            columns[col][d] += 1
            boxes[box_index(row, col)][d] += 1
            board[row][col] = str(d)
            
        def remove_number(d, row, col):
            """
            Remove a number which didn't lead 
            to a solution
            """
            del rows[row][d]
            del columns[col][d]
            del boxes[box_index(row, col)][d]
            board[row][col] = '.'    
            
        def place_next_numbers(row, col):
            """
            Call backtrack function in recursion
            to continue to place numbers
            till the moment we have a solution
            """
            # if we're in the last cell
            # that means we have the solution
            if col == N - 1 and row == N - 1:
                nonlocal sudoku_solved
                sudoku_solved = True
            #if not yet    
            else:
                # if we're in the end of the row
                # go to the next row
                if col == N - 1:
                    backtrack(row + 1, 0)
                # go to the next column
                else:
                    backtrack(row, col + 1)
                
                
        def backtrack(row = 0, col = 0):
            """
            Backtracking
            """
            # if the cell is empty
            if board[row][col] == '.':
                # iterate over all numbers from 1 to 9
                for d in range(1, 10):
                    if could_place(d, row, col):
                        place_number(d, row, col)
                        place_next_numbers(row, col)
                        # if sudoku is solved, there is no need to backtrack
                        # since the single unique solution is promised
                        if not sudoku_solved:
                            remove_number(d, row, col)
            else:
                place_next_numbers(row, col)
                    
        # box size
        n = 3
        # row size
        N = n * n
        # lambda function to compute box index
        box_index = lambda row, col: (row // n ) * n + col // n
        
        # init rows, columns and boxes
        rows = [defaultdict(int) for i in range(N)]
        columns = [defaultdict(int) for i in range(N)]
        boxes = [defaultdict(int) for i in range(N)]
        for i in range(N):
            for j in range(N):
                if board[i][j] != '.': 
                    d = int(board[i][j])
                    place_number(d, i, j)
        
        sudoku_solved = False
        backtrack()
        
        
class Solution:  # one solution
    def solveSudoku(self, board):
        N = 9

        # Use an array to record the status
        rows = [[0] * N for _ in range(N)]
        cols = [[0] * N for _ in range(N)]
        boxes = [[0] * N for _ in range(N)]
        queue = []

        for r in range(N):
            for c in range(N):
                # Check if the position is filled with number
                if board[r][c] == ".":
                    queue.append((r,c))
                else:
                    hash = int(board[r][c]) - 1
                    rows[r][hash] = 1
                    cols[c][hash] = 1
                    idx = (r // 3) * 3 + c // 3
                    boxes[idx][hash] = 1
        # if we want all results, do not early terminate it, copy the result            
        def dfs():
            if not queue:
                return True 
            x,y = queue.pop()
            for i in range(1, N+1):
                if rows[x][i-1] or cols[y][i-1] or boxes[(x//3)*3 + y//3][i-1]:
                    continue
                rows[x][i-1] = 1
                cols[y][i-1] = 1
                boxes[(x//3)*3 + y//3][i-1] = 1
                board[x][y] = str(i)
                if dfs():
                    return True 
                rows[x][i-1] = 0
                cols[y][i-1] = 0
                boxes[(x//3)*3 + y//3][i-1] = 0
                board[x][y] = "."
            queue.append((x,y))
        dfs()
        
        
import copy   
class AllSolution:  # all solutions
    def solveSudoku(self, board):
        N = 9

        # Use an array to record the status
        rows = [[0] * N for _ in range(N)]
        cols = [[0] * N for _ in range(N)]
        boxes = [[0] * N for _ in range(N)]
        queue = []  # need to fill

        for r in range(N):
            for c in range(N):
                # Check if the position is filled with number
                if board[r][c] == ".":
                    queue.append((r,c))
                else:
                    hash = int(board[r][c]) - 1  # record existing ones
                    rows[r][hash] = 1
                    cols[c][hash] = 1
                    idx = (r // 3) * 3 + c // 3
                    boxes[idx][hash] = 1
        # if we want all results, do not early terminate it, copy the result      
        result = []      
        def dfs():
            if not queue:
                result.append(copy.deepcopy(board))
                return
            x,y = queue.pop()
            for i in range(1, N+1):
                if rows[x][i-1] or cols[y][i-1] or boxes[(x//3)*3 + y//3][i-1]:
                    continue
                rows[x][i-1] = 1
                cols[y][i-1] = 1
                boxes[(x//3)*3 + y//3][i-1] = 1
                board[x][y] = str(i)
                dfs()
                rows[x][i-1] = 0
                cols[y][i-1] = 0
                boxes[(x//3)*3 + y//3][i-1] = 0
                board[x][y] = "."
            queue.append((x,y))
        dfs()
        for res in result:
            print(f"result: ")
            for row in res:
                print(row)
        
board1 = [["5","3",".",".","7",".",".",".","."],
         ["6",".",".","1","9","5",".",".","."],
         [".","9","8",".",".",".",".","6","."],
         ["8",".",".",".","6",".",".",".","3"],
         ["4",".",".","8",".","3",".",".","1"],
         ["7",".",".",".","2",".",".",".","6"],
         [".","6",".",".",".",".","2","8","."],
         [".",".",".","4","1","9",".",".","5"],
         [".",".",".",".","8",".",".","7","9"]]
board2 = [[".",".",".",".","7",".",".",".","."],
         [".",".",".","1","9","5",".",".","."],
         [".","9","8",".",".",".",".","6","."],
         ["8",".",".",".","6",".",".",".","3"],
         ["4",".",".","8",".","3",".",".","1"],
         ["7",".",".",".","2",".",".",".","6"],
         [".","6",".",".",".",".","2","8","."],
         [".",".",".","4","1","9",".",".","5"],
         [".",".",".",".","8",".",".","7","."]]
s = AllSolution()
s.solveSudoku(board2)  

# result: 
# ['3', '4', '5', '6', '7', '8', '9', '1', '2']
# ['6', '7', '2', '1', '9', '5', '3', '4', '8']
# ['1', '9', '8', '3', '4', '2', '5', '6', '7']
# ['8', '5', '9', '7', '6', '1', '4', '2', '3']
# ['4', '2', '6', '8', '5', '3', '7', '9', '1']
# ['7', '1', '3', '9', '2', '4', '8', '5', '6']
# ['9', '6', '1', '5', '3', '7', '2', '8', '4']
# ['2', '8', '7', '4', '1', '9', '6', '3', '5']
# ['5', '3', '4', '2', '8', '6', '1', '7', '9']
# result: 
# ['5', '3', '4', '6', '7', '8', '9', '1', '2']
# ['6', '7', '2', '1', '9', '5', '3', '4', '8']
# ['1', '9', '8', '3', '4', '2', '5', '6', '7']
# ['8', '5', '9', '7', '6', '1', '4', '2', '3']
# ['4', '2', '6', '8', '5', '3', '7', '9', '1']
# ['7', '1', '3', '9', '2', '4', '8', '5', '6']
# ['9', '6', '1', '5', '3', '7', '2', '8', '4']
# ['2', '8', '7', '4', '1', '9', '6', '3', '5']
# ['3', '4', '5', '2', '8', '6', '1', '7', '9']


result: 
['3', '4', '5', '6', '7', '8', '9', '1', '2']
['6', '7', '2', '1', '9', '5', '3', '4', '8']
['1', '9', '8', '3', '4', '2', '5', '6', '7']
['8', '5', '9', '7', '6', '1', '4', '2', '3']
['4', '2', '6', '8', '5', '3', '7', '9', '1']
['7', '1', '3', '9', '2', '4', '8', '5', '6']
['9', '6', '1', '5', '3', '7', '2', '8', '4']
['2', '8', '7', '4', '1', '9', '6', '3', '5']
['5', '3', '4', '2', '8', '6', '1', '7', '9']
result: 
['5', '3', '4', '6', '7', '8', '9', '1', '2']
['6', '7', '2', '1', '9', '5', '3', '4', '8']
['1', '9', '8', '3', '4', '2', '5', '6', '7']
['8', '5', '9', '7', '6', '1', '4', '2', '3']
['4', '2', '6', '8', '5', '3', '7', '9', '1']
['7', '1', '3', '9', '2', '4', '8', '5', '6']
['9', '6', '1', '5', '3', '7', '2', '8', '4']
['2', '8', '7', '4', '1', '9', '6', '3', '5']
['3', '4', '5', '2', '8', '6', '1', '7', '9']


In [ ]:
# 1152. Analyze User Website Visit Pattern


class Solution:
    def mostVisitedPattern(self, username: List[str], timestamp: List[int], website: List[str]) -> List[str]:
		
		# Create tuples as shown in description
		# The timestamps may not always be pre-ordered (one of the testcases)
		# Sort first based on user, then time (grouping by user)
		# This also helps to maintain order of websites visited in the later part of the solution
		
		users = defaultdict(list)
	    # It is not necessary to use defaultdict here, we can manually create dictionaries too
		
        for user, time, site in sorted(zip(username, timestamp, website), key = lambda x: (x[0],x[1])):    
            users[user].append(site)     # defaultdicts simplify and optimize code

        patterns = Counter()   # this can also be replaced with a manually created dictionary of counts
		
		# Get unique 3-sequence (note that website order will automatically be maintained)
		# Note that we take the set of each 3-sequence for each user as they may have repeats
		# For each 3-sequence, count number of users
		
        for user, sites in users.items():
            patterns.update(Counter(set(combinations(sites, 3))))     
            
        for user, sites in users.items():
    	
			three_sequence_combos = combinations(sites, 3)  # permutation
			three_sequence_combos = set(three_sequence_combos)    # avoid counting repeats
			
			three_sequence_combos = Counter(three_sequence_combos)     # convert to dictionary of counts
			patterns.update(three_sequence_combos)    # count the number of users having the same 3-sequence
		
		# Re-iterating above step for clarity
		# 1. first get all possible 3-sequences combinations(sites, 3)
		# 2. then, count each one once (set)
		# 3. finally, count the number of times we've seen the 3-sequence for every user (patterns.update(Counter)) 
		# - updating a dictionary will update the value for existing keys accordingly (int in this case)
		
		# An expanded version of the above step is given below.
			
    #         print(patterns)  # sanity check
	
		# get most frequent 3-sequence sorted lexicographically
        return max(sorted(patterns), key=patterns.get)
    
    
# self
import collections
class Solution:
    def mostVisitedPattern(self, username: List[str], timestamp: List[int], website: List[str]) -> List[str]:
        user2sites = collections.defaultdict(list)
        for user, _, site in sorted(zip(username, timestamp, website)):
            user2sites[user].append(site)
        pattern2score = collections.Counter()
        for user in user2sites:
            sites = user2sites[user]
            size = len(sites)
            pattern3set = set()
            for k in range(2, size):
                for j in range(1, k):
                    for i in range(j):
                        pattern3set.add((sites[i], sites[j], sites[k]))
            pattern2score.update(pattern3set)
        sorted_list = sorted(pattern2score.items(), key=lambda kv: (-kv[1], kv[0]))
        return sorted_list[0][0] if sorted_list else None
            
# Follow up: If you cannot store that data on a single machine,
# and you have very large data - how can you generate that on a distributed setup.         

# partition by user, and if random enough, each machine's result is good enough,
# if want to calculate exact, top 10 return from each partition, then get top 1          
        

In [ ]:
# 1235. Maximum Profit in Job Scheduling

class Solution:
    def jobScheduling(self, startTime: List[int], endTime: List[int], profit: List[int]) -> int:
        
        data = sorted(zip(startTime, endTime, profit), key = lambda x:x[1])
        
        #dp[i] is the max_profit with endtime at dp_end_time[i]
        dp_profit = [0]   # increasing profit value, added by endTime ordered way
        dp_end_time = [0] # matched end time value for above profit array
        
        for i, e in enumerate(data):
            start, end, profit = e
            
            idx = bisect.bisect_left(dp_end_time, start + 1)
            # idx is first end time >= start + 1
            # idx - 1 is the last(max since increasing) end time <= start
            
            curr_profit = dp_profit[idx - 1] + profit
            
            if curr_profit > dp_profit[-1]:  # always add profit and end at the same time at the end
                dp_profit.append(curr_profit)
                dp_end_time.append(end)
                
        return dp_profit[-1]
    
    
    # 26 ^ 20 ~ 2 ^9 ^20
    
# self
class Solution:
    def jobScheduling(self, startTime: List[int], endTime: List[int], profit: List[int]) -> int:
        
        data = sorted(zip(endTime, startTime, profit))
        
        #dp[i] is the max_profit with endtime at dp_end_time[i]
        dp_profit = [0]   # increasing profit value, added by endTime ordered way
        dp_end_time = [0] # matched end time value for above profit array
        
        for e in data:
            end, start,  profit = e
            
            idx = bisect.bisect_left(dp_end_time, start + 1)  # we can also use start here and test idx-1 and idx
            # idx is first end time >= start + 1
            # idx - 1 is the last(max since increasing) end time <= start
            
            curr_profit = dp_profit[idx - 1] + profit
            
            if curr_profit > dp_profit[-1]:  # always add profit and end at the same time at the end
                dp_profit.append(curr_profit)
                dp_end_time.append(end)  # add additional array to remember (start, end, profict, previous idx)
                
        return dp_profit[-1]
    
# all the paths??? too difficult !!!
    
# You're a dasher, and you want to try planning out your schedule. 
# You can view a list of deliveries along with their associated start time, end time, and dollar amount
# for completing the order. Assuming dashers can only deliver one order at a time, 
# determine the maximum amount of money you can make from the given deliveries.

# The inputs are as follows:

# int start_time: when you plan to start your schedule
# int end_time: when you plan to end your schedule
# int d_starts[n]: the start times of each delivery[i]
# int d_ends[n]: the end times of each delivery[i]
# int d_pays[n]: the pay for each delivery[i]
# The output should be an integer representing the maximum amount of money you can make by forming a schedule with the given deliveries.

# Example #1
# start_time = 0
# end_time = 10
# d_starts = [2, 3, 5, 7]
# d_ends = [6, 5, 10, 11]
# d_pays = [5, 2, 4, 1]
# Expected output: 6

import bisect
def max_profit(start_time, end_time, d_starts, d_ends, d_pays):
    dp_end = [0]
    dp_profit = [0]
    dp_track = [0]
    for end, start, pay in sorted(zip(d_ends, d_starts, d_pays)):
        if end > end_time or start < start_time:
            continue
        idx = bisect.bisect_left(dp_end, start+1)
        profit = dp_profit[idx-1] + pay
        if profit > dp_profit[-1]:
            dp_profit.append(profit)
            dp_end.append(end)
            dp_track.append((start, end, pay, idx-1))
            
    previous = len(dp_track)-1
    path = []
    while previous > 0:
        path.append(dp_track[previous])
        previous = dp_track[previous][-1]
    print(path[::-1])
    print(dp_track)
    print(dp_profit)
    print(dp_end)
    return dp_profit[-1]

start_time = 0
end_time = 10
d_starts = [2, 3, 5, 7]
d_ends = [6, 5, 10, 11]
d_pays = [5, 2, 4, 1]

print(max_profit(start_time, end_time, d_starts, d_ends, d_pays))
        
# path: [(3, 5, 2, 0), (5, 10, 4, 1)]
# dp_track: [0, (3, 5, 2, 0), (2, 6, 5, 0), (5, 10, 4, 1)]
# dp_profit: [0, 2, 5, 6]
# dp_end: [0, 5, 6, 10]
# max: 6

In [99]:
# n dice, 6 faces, 1-6, get all results

def dice_combo(n):
    if n < 1:
        return []
    result = [[]]
    for _ in range(n):
        new_result = []
        for res in result:
            for i in range(1,7):
                new_result.append(res+[i])
        result = new_result
    return result 

# time O(6^N)
# space O(6^N)

print(dice_combo(2))
                

[[1, 1], [1, 2], [1, 3], [1, 4], [1, 5], [1, 6], [2, 1], [2, 2], [2, 3], [2, 4], [2, 5], [2, 6], [3, 1], [3, 2], [3, 3], [3, 4], [3, 5], [3, 6], [4, 1], [4, 2], [4, 3], [4, 4], [4, 5], [4, 6], [5, 1], [5, 2], [5, 3], [5, 4], [5, 5], [5, 6], [6, 1], [6, 2], [6, 3], [6, 4], [6, 5], [6, 6]]


In [110]:
 # A DashMart is a warehouse run by DoorDash that houses items found in
 # convenience stores, grocery stores, and restaurants. We have a city with open
 # roads, blocked-off roads, and DashMarts.
 #
 # City planners want you to identify how far a location is from its closest
 # DashMart.
 #
 # You can only travel over open roads (up, down, left, right).
 #
 # Locations are given in [row, col] format.
 #
 # Example:
 #
 # [
 # # 0 1 2 3 4 5 6 7 8
 # ['X', ' ', ' ', 'D', ' ', ' ', 'X', ' ', 'X'], # 0
 # ['X', ' ', 'X', 'X', ' ', ' ', ' ', ' ', 'X'], # 1
 # [' ', ' ', ' ', 'D', 'X', 'X', ' ', 'X', ' '], # 2
 # [' ', ' ', ' ', 'D', ' ', 'X', ' ', ' ', ' '], # 3
 # [' ', ' ', ' ', ' ', ' ', 'X', ' ', ' ', 'X'], # 4
 # [' ', ' ', ' ', ' ', 'X', ' ', ' ', 'X', 'X'] # 5
 # ]
 #
 # ' ' represents an open road that you can travel over in any direction (up, down, left, or right).
 # 'X' represents an blocked road that you cannot travel through.
 # 'D' represents a DashMart.
 #
 # # list of pairs [row, col]
 # locations = [
 # [2, 2],
 # [4, 0],
 # [0, 4],
 # [2, 6],
 # ]
 #
 # answer = [1, 4, 1, 5]
 #
 # Implement Function:
 # - get_closest_dashmart(city, locations)
 #
 # Provided:
 # - city: List[str]
 # - locations: List[List[int]]
 #
 # Return:
 # - answer: List[int]
 
 
def find_closest_dashmart(grid, locations):
    if not grid or not locations:
        return None 
    H = len(grid)
    W = len(grid[0])
    queue = []
    targets = set([(x,y) for x,y in locations])
    # print(targets)
    level = 0
    visited = set()
    loc2dist = {}
    cur2pre = {}
    for i in range(H):
        for j in range(W):
            if grid[i][j] == 'D':
                queue.append((i,j))
                visited.add((i,j))

    while queue:
        if len(locations) == len(loc2dist):
            break 
        new_queue = []
        # print(queue)
        for i, j in queue:
            if (i,j) in targets:
                loc2dist[i,j] = level
                # early termination
                if len(locations) == len(loc2dist):
                    break 
            for di, dj in (0,1), (0,-1), (1,0), (-1,0):
                ni = i + di
                nj = j + dj 
                if ni <0 or ni>H-1 or nj<0 or nj>W-1 or (ni, nj) in visited or grid[ni][nj]!=' ':
                    continue 
                visited.add((ni, nj))
                new_queue.append((ni, nj))
                cur2pre[ni, nj] = (i, j)
        queue = new_queue 
        level += 1
    result = []
    all_paths = []
    # print(loc2dist)
    for x,y in locations:
        result.append(loc2dist.get((x,y), -1))
        path = []
        while grid[x][y] != 'D':
            path.append((x,y))
            x,y = cur2pre[x,y]
        path.append((x,y))
        all_paths.append(path)
    print(all_paths)
    return result 

grid =[
 # 0 1 2 3 4 5 6 7 8
 ['X', ' ', ' ', 'D', ' ', ' ', 'X', ' ', 'X'], # 0
 ['X', ' ', 'X', 'X', ' ', ' ', ' ', ' ', 'X'], # 1
 [' ', ' ', ' ', 'D', 'X', 'X', ' ', 'X', ' '], # 2
 [' ', ' ', ' ', 'D', ' ', 'X', ' ', ' ', ' '], # 3
 [' ', ' ', ' ', ' ', ' ', 'X', ' ', ' ', 'X'], # 4
 [' ', ' ', ' ', ' ', 'X', ' ', ' ', 'X', 'X'] # 5
 ]
 
 locations = [
 [2, 2],
 [4, 0],
 [0, 4],
 [2, 6],
 ]


print(find_closest_dashmart(grid, locations))
        
            
    

[[(2, 2), (2, 3)], [(4, 0), (3, 0), (3, 1), (3, 2), (3, 3)], [(0, 4), (0, 3)], [(2, 6), (1, 6), (1, 5), (0, 5), (0, 4), (0, 3)]]
[1, 4, 1, 5]


In [ ]:
# At DoorDash, many deliveries are scheduled well in advance. To improve our assignment rate,
# we want to enable dashers to claim these scheduled deliveries early. 
# However, we noticed that certain dashers perform better, and want to reward them with a better selection. 
# As a simple solution, we will introduce open windows for when deliveries will appear for a particular dasher. 
# Below are the following requirements.
# deliveries scheduled two days or further into the future should never be available
# high tier dashers can see all of next day deliveries if the current time is 18:00 or later
# all dashers can see all of next day deliveries if the current time is 19:00 or later
# all dashers can see same day deliveries anytime
from datetime import datetime, timedelta


class Delivery(object):
    def __init__(self, idx, pickup_time, store_id):
        self.id = idx
        self.pickup_time = pickup_time
        self.store_id = store_id

class Dasher(object):
    def __init__(self, idx, tier):
        self.id = idx
        self.tier = tier # 'low', 'high'

def get_available_deliveries(dasher, deliveries, current_time):
    start = current_time 
    # current = datetime.now()  # get server time
    today = datetime(current_time.year, current_time.month, current_time.day)
    time_for_low = today + timedelta(hours=19)
    time_for_high = today + timedelta(hours=18)
    end = today + timedelta(days=1)
    if current_time >= time_for_low:
        end += timedelta(days=1)
    elif current_time >= time_for_high and dasher.tier == "high":
        end += timedelta(days=1)
    result = []
    print(start, end)
    for d in deliveries:
        print(d.pickup_time)
        if start > d.pickup_time or d.pickup_time > end:
            continue
        result.append(d)
    return result 
    


# Sample test.
today = datetime(2021, 1, 15)
dasher = Dasher('dasher', 'high')
deliveries = [
Delivery('1', today + timedelta(hours=10), 'store_1'),
Delivery('2', today + timedelta(hours=30), 'store_1')
]
available_deliveries = get_available_deliveries(
    dasher=dasher,
    deliveries=deliveries,
    current_time=today + timedelta(hours=18)
)
print([d.id for d in available_deliveries]) # Should include delivery 1.


# from datetime import datetime, timedelta  # do not trust client time, user server time
# current = datetime.now()
# print(current + timedelta(hours=1))


# from datetime import datetime, timedelta
# today = datetime(2021, 1, 15)
# pm6 = today + timedelta(hours=13)
# print(type(today.year))
# print(type(today.month))
# print(type(today.day))
# redo = datetime(pm6.year, pm6.month, pm6.day)
# print(redo)

In [ ]:
# We want to implement an in-memory tree key value store for Doordash Restaurant Menus.
# Definitions:

# path is a / separate string describing the node. Example /Tres Potrillos/tacos/al_pastor
# Values are all strings
# API spec:
# get(path): String -> returns the value of the node at the given path
# create(path, value) -> creates a new node and sets it to the given value.
# Should error out if the node already exists or if the node's parent does not exist. 
# That is /Sweetgreen/naan_roll cannot be created if /Sweetgreen has not already been created
# delete(path) -> deletes a node, but ONLY if it has no children


# 1166. Design File System


# The TrieNode data structure.
class TrieNode(object):
    def __init__(self, name):
        self.map = defaultdict(TrieNode)
        self.name = name
        self.value = -1

class FileSystem:

    def __init__(self):
        
        # Root node contains the empty string.
        self.root = TrieNode("")

    def createPath(self, path: str, value: int) -> bool:
        
        # Obtain all the components
        components = path.split("/")
        
        # Start "curr" from the root node.
        cur = self.root
        
        # Iterate over all the components.
        for i in range(1, len(components)):
            name = components[i]
            
            # For each component, we check if it exists in the current node's dictionary.
            if name not in cur.map:
                
                # If it doesn't and it is the last node, add it to the Trie.
                if i == len(components) - 1:
                    cur.map[name] = TrieNode(name)
                else:
                    return False
            cur = cur.map[name]
        
        # Value not equal to -1 means the path already exists in the trie. 
        if cur.value!=-1:
            return False
        
        cur.value = value
        return True

    def get(self, path: str) -> int:
        
        # Obtain all the components
        cur = self.root
        
        # Start "curr" from the root node.
        components = path.split("/")
        
        # Iterate over all the components.
        for i in range(1, len(components)):
            
            # For each component, we check if it exists in the current node's dictionary.
            name = components[i]
            if name not in cur.map:
                return -1
            cur = cur.map[name]
        return cur.value
    
# self
import re
class Node:
    def __init__(self, value):
        self.value = value 
        self.children = {}  # key is the path, value is next node

class FileSystem:
    
    def __init__(self):
        self.root = Node(-1)   
        self.callback = None 
        
    
    def watch(self, path, callback_func):
        parent, current = self._get_parent_current(path)
        if not parent or current not in parent.children:
            return -1
        parent.children[current].callback = callback_func
        
      

    def createPath(self, path: str, value: int) -> bool:
        parent, current = self._get_parent_current(path)
        if not parent or current in parent.children:
            return False
        parent.children[current] = Node(value)
        self.callback('create', path, nil, value)
        return True 
        
    def deletePath(self, path: str) -> bool:
        parent, current = self._get_parent_current(path)
        if not parent or current not in parent.children:
            return False
        cur_node = parent.children[current]
        if len(cur_node.children) > 0:
            return False
        parent.children.pop(current, None)
        self.callback('delete', path, value, nil)
        return True    

    def get(self, path: str) -> int:
        parent, current = self._get_parent_current(path)
        if not parent or current not in parent.children:
            return -1
        self.callback('get', path, value, nil)
        return parent.children[current].value
    
    
            
    def _parse(self, path):
        path = re.sub(r"\s+", '', path)
        if not path or not path.startswith('/'):
            return None 
        parts = path.split('/')[1:]
        return parts 
    
    
    def _get_parent_current(self, path):
        parts = self._parse(path)
        if not parts:
            return None, None
        cur = self.root
        for part in parts[:-1]:
            if part in cur.children:
                cur = cur.children[part]
            else:
                return None, None
        return cur, parts[-1]
        


# Your FileSystem object will be instantiated and called as such:
# obj = FileSystem()
# param_1 = obj.createPath(path,value)
# param_2 = obj.get(path)


# def callbackFunc(s):
#     print('Length of the text file is : ', s)

# def printFileLength(path, callback):
#     f = open(path, "r")
#     length = len(f.read())
#     f.close()
#     callback(length)

# if __name__ == '__main__':
#     printFileLength("sample.txt", callbackFunc)





In [ ]:
# At DoorDash, menus are updated daily even hourly to keep them up-to-date. 
# Each menu can be regarded as a tree. When the merchant sends us the latest menu, can we calculate how many nodes has changed?

# Assume each Node structure is as below:

# class Node {
# String key;
# int value;
# boolean active;
# List children;
# }
# Assume there are no duplicate nodes with the same key.

# Output: Return the number of changed nodes in the tree.

# Example 1
# Existing Menu in our system:

# Existing tree
#           a(1, T)
#           /  \
#       b(2, T) c(3, T)
#       / \      \
# d(4, T) e(5, T) f(6, T)
# ( Legend - a(1, T) a is the key, 1 is the value, T is True for active status )

# New Menu sent by the Merchant:

# New tree
# a(1, T)
#  \
# c(3, F)
#   \
# f(66, T)
# Expected Answer: 5 Explanation: Node b, Node d, Node e are automatically set to inactive.
# The active status of Node c and the value of Node f changed as well.

# Example 2
# Existing Menu in our system:

# Existing tree
#       a(1, T)
#       /       \
#   b(2, T)     c(3, T)
#       / \         \
# d(4, T) e(5, T) g(7, T)
# New Menu sent by the Merchant:

# New tree
#            a(1, T)
#           /       \
#       b(2, T)       c(3, T)
#       /   |   \          \
# d(4, T) e(5, T) f(6, T)   g(7, F)
# Expected Answer: 2 Explanation: Node f is a newly-added node. Node g changed from Active to inactive

class Node:
    def __init__(self, value, active):
        self.value = value 
        self.active = active # true / false
        self.children = {}
        
class TreeMenu:
    def __init__(self):
        self.root = Node(-1, False)
        
    def tree2dict(self):
        result = {}
        self.dfs(self.root, [], result)
        return result 
    
    def dfs(self,root, path, path2value):
        cur_path = "/"+"/".join(path)
        path2value[cur_path] = (root.value, root.active)
        for child in root.children:
            path.append(child)
            self.dfs(root.children[child], path, path2value)
            path.pop()
            
    def totalDiff(self, another):
        thisDic = self.tree2dict()
        thatDic = another.tree2dict()
        thisSet = set(thisDic.keys())
        thatSet = set(thatDic.keys())
        miss_add = thisSet ^ thatSet
        common  = thisSet & thatSet
        count = len(miss_add)
        for path in common:
            if thisDic[path] == thatDic[path]:
                continue
            count += 1
        return count 
    
def cntNode(root): # count nodes including root
        count = 1
        for child in root.children:
            count += cntNode(root.children[child])
        return count
     
def treeDiff(old_tree, new_tree):
    
    def dfs(root1, root2):  # dfs same key node
        diff = 0
        if root1.value!=root2.value or root1.active!=root2.active:
            diff += 1
        for child1 in root1.children:
            if child1 not in root2.children:
                # print(f"count: {child1}")
                diff += cntNode(root1.children[child1])
                # print(f"count: {child1} -> {diff}")
            else:
                # print(f"dfs: {child1}")
                diff += dfs(root1.children[child1], root2.children[child1])
                # print(f"dfs: {child1} -> {diff}")
            
        for child2 in root2.children:
            if child2 not in root1.children:
                # print(f"count2: {child2}")
                diff += cntNode(root2.children[child2])
                # print(f"count2: {child2} -> {diff}")
                
        return diff 
    
    if not old_tree or not new_tree:
        return 0
    return dfs(old_tree.root, new_tree.root)    
        
        
m1 = TreeMenu()  
a = Node(1, True)
m1.root.children['a'] = a   
b = Node(2, True)
c = Node(3, True)
d = Node(4, True)
e = Node(5, True)
f = Node(6, True)
a.children['b'] = b  
a.children['c'] = c        
b.children['d'] = d  
b.children['e'] = e        
c.children['f'] = f

m2 = TreeMenu()  
aa = Node(1, True)
m2.root.children['a'] = aa  

cc = Node(3, False)

ff = Node(66, True)  
aa.children['c'] = cc        
      
cc.children['f'] = ff

print(m1.totalDiff(m2))

print(treeDiff(m2,m1))


In [ ]:
# 992. Subarrays with K Different Integers


class Solution:
    def subarraysWithKDistinct(self, A: List[int], K: int) -> int:
        
        res = 0        
        map1 = {}
        map2 = {}
        slow1 = 0
        slow2 = 0
        for fast in range(len(A)):
            
            map1[A[fast]] = map1.get(A[fast], 0) + 1
            map2[A[fast]] = map2.get(A[fast], 0) + 1

            while len(map1) > K:
                map1[A[slow1]] -= 1
                if map1[A[slow1]] == 0:
                    del map1[A[slow1]]
                slow1 += 1
            # stop point of slow1 is len(map1) == k
            
            while len(map2) > K:
                map2[A[slow2]] -= 1
                if map2[A[slow2]] == 0:
                    del map2[A[slow2]]
                slow2 += 1
            # stop point of slow1 is len(map1) == k - 1
            
                
            res += (slow2 - slow1)
            
        return res 
    
# using at most method

class Solution:
    def subarraysWithKDistinct(self, A: List[int], K: int) -> int:
        
        def atMost(K):
            res = 0        
            map = {}
            slow = 0
            for fast in range(len(A)):
                map[A[fast]] = map.get(A[fast], 0) + 1
                while len(map) > K:
                    map[A[slow]] -= 1
                    if map[A[slow]] == 0:
                        del map[A[slow]]
                    slow += 1
                # stop point of slow is len(map) == k
                res += fast - slow + 1
            return res
    
        return atMost(K) - atMost(K - 1)


In [ ]:
# 1472. Design Browser History


class BrowserHistory:
    
    def __init__(self, homepage: str):
        self.history = [homepage]
        self.curr = 0
        self.bound = 0

    def visit(self, url: str) -> None:
        self.curr += 1
        if self.curr == len(self.history):
            self.history.append(url)
        else:
            self.history[self.curr] = url
        self.bound = self.curr

    def back(self, steps: int) -> str:
        self.curr = max(self.curr - steps, 0)
        return self.history[self.curr]

    def forward(self, steps: int) -> str:
        self.curr = min(self.curr + steps, self.bound)
        return self.history[self.curr]
    
# self.

class BrowserHistory:
    
    def __init__(self, homepage: str):
        self.history = [homepage]
        self.curr = 0
        self.bound = 0  # max index

    def visit(self, url: str) -> None:
        self.curr += 1
        if self.curr == len(self.history):
            self.history.append(url)
        else:
            self.history[self.curr] = url
        self.bound = self.curr

    def back(self, steps: int) -> str:
        self.curr = max(self.curr - steps, 0)
        return self.history[self.curr]

    def forward(self, steps: int) -> str:
        self.curr = min(self.curr + steps, self.bound)
        return self.history[self.curr]

In [ ]:
# 859. Buddy Strings
# test if we can swap 2 chars in s to get goal

class Solution:
    def buddyStrings(self, s: str, goal: str) -> bool:
        
        if len(s) != len(goal): 
            return False
        
        if s == goal:
            seen = set()
            for a in s:
                if a in seen:
                    return True
                seen.add(a)
            return False

        pairs = []
        for a, b in zip(s, goal):
            if a != b:
                pairs.append((a, b))
            if len(pairs) >= 3: 
                return False
        return len(pairs) == 2 and pairs[0] == pairs[1][::-1]
    
  
# 854. K-Similar Strings  
    
def kSimilarity(A, B):
    def nei(x):
		i = 0
		while x[i] == B[i]: i+=1
		for j in range(i+1, len(x)):
			if x[j] == B[i]: yield x[:i]+x[j]+x[i+1:j]+x[i]+x[j+1:]
	q, seen = [(A,0)], {A}
	for x, d in q:
		if x == B: return d
		for y in nei(x):
			if y not in seen:
				seen.add(y), q.append((y,d+1))
    
# self
class Solution:
    def kSimilarity(self, source: str, target: str) -> int:
        def get_next(ss): # get next string closer to target
            for i in range(len(ss)):
                if ss[i] != target[i]:
                    break
            
            for j in range(i+1, len(ss)):
                if target[i] == ss[j]:    # ss swap i and j
                    yield ss[:i] + ss[j] + ss[i+1:j] + ss[i] + ss[j+1:]
        
        seen = {source}
        queue = [source]
        level = 0
        while queue:
            new_queue = []
            for node in queue:
                if node == target:
                    return level 
                for nb in get_next(node):
                    if nb not in seen:
                        new_queue.append(nb)
                        seen.add(nb)
            level += 1
            queue = new_queue 
        return None 
            

In [ ]:
# 1347. Minimum Number of Steps to Make Two Strings Anagram
# 

class Solution:
    def minSteps(self, s: str, t: str) -> int:
        count1 = collections.Counter(s)
        count2 = collections.Counter(t)
        
        totalCount = 0
        for i in range(26):
            c = chr(i + ord('a'))
            totalCount += abs(count1[c] - count2[c])
        return totalCount // 2

In [ ]:
# 124. Binary Tree Maximum Path Sum

# a more concise solution
class Solution:
    def maxPathSum(self, root: TreeNode) -> int:
        
        self.ans = -float('inf')
        def help(node):
            
            if not node:
                return 0
#            if not node.left and not node.right:
#                return node.val
            
#          
            leftSum = max(help(node.left), 0)         
            rightSum = max(help(node.right), 0)
           # four different conditions to form a path 
            
            # if leftSum and rightSum are both negative, self.ans will make max(self.ans, node.val)
            self.ans = max(self.ans, leftSum + rightSum + node.val)
            
            nonEmptyPathMax = max(leftSum, rightSum) + node.val
            
            
            return nonEmptyPathMax
        
        help(root)
        return self.ans


# self


# Definition for a binary tree node.
# class TreeNode:
#     def __init__(self, val=0, left=None, right=None):
#         self.val = val
#         self.left = left
#         self.right = right
class Solution:
    def maxPathSum(self, root: Optional[TreeNode]) -> int:
        
        def helper(root):
            if not root:
                return float('-inf'), float('-inf')
            left_single, left_double = helper(root.left)
            right_single, right_double = helper(root.right)
            single = max(left_single+root.val, right_single + root.val, root.val)
            double = max(left_double, right_double, root.val + left_single + right_single, single)
            return single, double    
        return helper(root)[1]
        
        
# leaf to leaf, active or not leaf
class Solution:
    def maxPathSum(self, root):
        if not root:
            return 0
        def helper(root):
            if not root:
                return 0, float('-inf')
            left_single, left_double = helper(root.left) 
            right_single, right_double = helper(root.right)
            single = max(left_single+root.val, right_single + root.val)
            double = max(left_double, right_double, root.val + left_single + right_single)
            return single, double    
        return helper(root)[1]
    
# leaf to leaf
class Solution:
    def maxPathSum(self, root):
        if not root:
            return 0
        def helper(root):
            if not root:
                return float('-inf'), float('-inf')
            if not root.left and not root.right:
                return root.val, root.val 
            left_single, left_double = helper(root.left) 
            right_single, right_double = helper(root.right) 
            single = max(left_single+root.val, right_single + root.val)
            double = max(left_double, right_double, root.val + left_single + right_single)
            return single, double    
        return helper(root)[1]


# negative number, already considered



# sub sets as start and end ??? 
class Solution:
    def maxPathSum(self, root, nodes):
        if not root or not nodes:
            return 0
        def helper(root):
            if not root:
                return float('-inf'), float('-inf')
            if root in nodes:
                return root.val, root.val 
            left_single, left_double = helper(root.left) 
            right_single, right_double = helper(root.right) 
            single = max(left_single+root.val, right_single + root.val)
            double = max(left_double, right_double, root.val + left_single + right_single)
            return single, double    
        return helper(root)[1]


# 

In [ ]:
# 84. Largest Rectangle in Histogram

class Solution:
    def largestRectangleArea(self, heights: List[int]) -> int:
        def calculateArea(heights: List[int], start: int, end: int) -> int:
            if start > end:
                return 0
            min_index = start
            for i in range(start, end + 1):
                if heights[min_index] > heights[i]:
                    min_index = i
            return max(
                heights[min_index] * (end - start + 1),
                calculateArea(heights, start, min_index - 1),
                calculateArea(heights, min_index + 1, end),
            )

        return calculateArea(heights, 0, len(heights) - 1)
    
# self
# min stack or queue should work as well

def largestRectangleArea(self, height):
    height.append(0)
    stack = [-1]
    ans = 0
    for i in xrange(len(height)):
        while height[i] < height[stack[-1]]:
            h = height[stack.pop()]
            w = i - stack[-1] - 1
            ans = max(ans, h * w)
        stack.append(i)
    height.pop()
    return ans

In [ ]:
# 826. Most Profit Assigning Work
class Solution(object):
    def maxProfitAssignment(self, difficulty, profit, worker):
        jobs = zip(difficulty, profit)
        jobs.sort()
        ans = i = best = 0
        for skill in sorted(worker):
            while i < len(jobs) and skill >= jobs[i][0]:
                best = max(best, jobs[i][1])
                i += 1
            ans += best
        return ans




In [ ]:
# 45. Jump Game II


class Solution:
    def jump(self, nums: List[int]) -> int:
        
        queue = collections.deque([(0, 0)]) # position arrived and how many jumps so far
        visited = set([0])
        while queue:
            pos, d = queue.popleft()
            
            if pos == len(nums) - 1:
                return d
            
            for new_pos in range(pos, pos + nums[pos] + 1):
                if new_pos < len(nums) and new_pos not in visited:
                    visited.add(new_pos)
                    queue.append((new_pos, d + 1))

In [ ]:
# 1345. Jump Game IV

class Solution:
    def minJumps(self, arr: List[int]) -> int:
        
        value_to_pos = collections.defaultdict(list)
        for i, val in enumerate(arr):
            value_to_pos[val].append(i)
            
            
        queue = collections.deque([(0, 0)])
        visited = set([0])
        while queue:
            node, d = queue.popleft()
            
            if node == len(arr) - 1:
                return d
            for nb in value_to_pos[arr[node]]:
                if nb not in visited:
                    queue.append((nb, d + 1))
                    visited.add(nb)
            
            # critical optimization here. All nodes are in visited and will not be used any more, it is safe to remove it
            value_to_pos[arr[node]].clear()
                    
            if node + 1 < len(arr) and node + 1 not in visited:
                queue.append((node + 1, d + 1))
                visited.add(node + 1)

            if node - 1 >= 0 and node - 1 not in visited:
                queue.append((node - 1, d + 1))
                visited.add(node - 1)


In [ ]:
# 735. Asteroid Collision


class Solution:
    def asteroidCollision(self, asteroids: List[int]) -> List[int]:
        
        stack = []
        
        for e in asteroids:
            
            if e > 0:
                stack.append(e)
            elif e < 0:
                while stack and 0< stack[-1] < -e:
                    stack.pop()
                
                if not stack or stack[-1] < 0:
                    stack.append(e)
                elif stack and stack[-1] == -e:
                    stack.pop()
        
        return stack

In [ ]:
# unlimited backpack

def unboundedBackpack(wt, V, m):
    
    f = [0] * (m + 1)
    for j in range(m + 1): # quota
        for i in range(len(wt)):  # items (unlimited)  
            if j >= wt[i]:
                f[j] = max(f[j], f[j - wt[i]] + V[i])
    print(f)         
    return f[-1]

W = 100
val = [10, 30, 20] 
wt = [5, 10, 15] 


# 0/1 backpack

def backpackII(A, V, W):
    
    N = len(A)
    
    dp = [[0] * (W + 1) for _ in range(N)]
    
    for i in range(N): # available items (max once picked)
        
        for j in range(W + 1): # quota
            
            if i == 0:
                dp[i][j] = V[i] if j >= A[0] else 0
            
            if j - A[i] >= 0:
                dp[i][j] = max(dp[i - 1][j], dp[i - 1][j - A[i]] + V[i])
            else:
                dp[i][j] = dp[i - 1][j]
                
    
    answer = dp[-1][-1]

In [ ]:
# 973. K Closest Points to Origin


